<a href="https://colab.research.google.com/github/SilasAppel/CDA-Class-2/blob/main/04_CDA_HH_midterm_synthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cultural Data Analysis

Synthesis
Week 01 - Week 04

In [1]:
# import necessary libraries
import os, re, csv
import numpy as np
import pandas as pd

## Loading the datasets: heritage homes webistes

The dataset is stored in a shared google drive:
https://drive.google.com/drive/folders/11Shm0edDOiWrOe56fzJQRZi-v_BPSW8E?usp=drive_link

Add it to your drive.

To access it, load your gdrive in 'Files' (see left pane of the notebook in google colab) and navigate to the shared folder. You may need to click on 'refresh' to make it appear on the list.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Load the github repository files where this notebook is stored to the available files. This will make it easier to import stopwords, url lists and other additional data we need.

In [ ]:
!git clone https://github.com/jazoza/cultural-data-analysis

## Import all datasets (4 countries)

You will have all datasets available for analysis and comparison, mapped in the following way:

> df0 - Dutch dataset

> df1 - UK dataset

> df2 - German dataset

> df3 - French dataset

In [ ]:
# Country code: change here between 'NL' and 'UK'
cc_list = ['NL', 'UK', 'DE', 'FR']

In [ ]:
gdrive_path = '/content/gdrive/MyDrive/CDA/'

In [ ]:
# Import scraped json data into 4 separate dataframes
df0=pd.read_json(gdrive_path+cc_list[0]+'_dataset_website-content-crawler.json')
# select columns for analysis: url, text, metadata
df0=df0[['url','text','metadata']]

df1=pd.read_json(gdrive_path+cc_list[1]+'_dataset_website-content-crawler.json')
# select columns for analysis: url, text, metadata
df1=df1[['url','text','metadata']]

df2=pd.read_json(gdrive_path+cc_list[2]+'_dataset_website-content-crawler.json')
# select columns for analysis: url, text, metadata
df2=df2[['url','text','metadata']]

df3=pd.read_json(gdrive_path+cc_list[3]+'_dataset_website-content-crawler.json')
# select columns for analysis: url, text, metadata
df3=df3[['url','text','metadata']]

df0.head()


Join all pages from a domain to an entry in the analysis. To do this, add a new column which will contain only the main domain name.

In [ ]:
# function to extract the main domain from the url in the dataset
def extract_main_domain(url):
    if not isinstance(str(url), str):
        print('NOT VALID',url)
        return None
    match = re.findall('(?:\\w+\\.)*\\w+\\.\\w*', str(url)) #'www\.?([^/]+)'
    return match[0].lstrip('www.') if match else None

In [ ]:
# Add a new column 'domain' and fill it by applying the extract_main_domain function to the 'url' column

# first, create a mapping of dataframes which could be addressed in a loop
df_dict = {'0':df0, '1':df1, '2':df2, '3':df3}

# then, loop through the df_dict to update each dataframe
for k, v in df_dict.items():
  cc_column = cc_list[int(k[-1])]+' domains'
  cc = cc_list[int(k[-1])]
  # print(cc_column, cc)
  urls = pd.read_csv(gdrive_path+'url_lists/'+cc_list[int(k[-1])]+'_urls.csv')[cc_column].values.tolist()
  domains = {extract_main_domain(url) for url in urls if extract_main_domain(url) is not None}
  matching_links = [link for link in v.url if extract_main_domain(link) in domains]
  # update the dataframe
  v['domain'] = v['url'].apply(extract_main_domain)

# check one of the dataframes
df1.head()

## Prepare the analysis

Import stopwords dictionaries for the 4 langauges we work with.
It is good to import all of them in our case, because many websites have sections is English, German or French even when this is not the main language of the website.

In [ ]:
# load a list of 'stopwords' function
def get_stopwords_list(stop_file_path):
    """load stop words """
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [ ]:
# Get the stopwords list for all languages (using cc_list previously defined)
# cc_list = ['NL', 'UK', 'DE', 'FR'] # remove the hashtag from this line to uncomment this code and make it run

stopwords = [] # empty list to which a list of stopwords will be appended in loop

for i in range(len(cc_list)):
  stopwords_cc_path = "/content/cultural-data-analysis/stopwords_archive/"+cc_list[i]+".txt"
  stopwords_cc = get_stopwords_list(stopwords_cc_path)
  #print(len(stopwords_cc)) # print how many words are in the list
  stopwords.extend(stopwords_cc)

#print(len(stopwords)) # print how many words are in all stopwords lists

In [ ]:
# you may need to include additional words which you notice as too frequent
special_stop_words = ['nbsp', 'nl', 'fr', 'de', 'uk', 'com', 'www', 'lit', ' '] # these might appear frequently as 'terms' in the corpus, so it's good to filter them
stopwords_ext = stopwords+special_stop_words

## 1. Term frequency

The cells below will compute a term-matrix and calculate the frequency of each unique word (token) in the corpus

This can be done for ALL words in the corpus, or ALL MEANINGFUL words (without so-called stop-words like 'the' or 'het')

In [ ]:
# CALCULATE TERM FREQUENCY OF ALL TERMS
from sklearn.feature_extraction.text import CountVectorizer

# convert the text documents into a matrix of token (word) counts
cvec_all = CountVectorizer().fit(df0.text) #### CHANGE df0 TO THE DATAFRAME YOU ANALYSE
df_matrix_all = cvec_all.transform(df0.text) #### CHANGE df0 TO THE DATAFRAME YOU ANALYSE
df_all = np.sum(df_matrix_all,axis=0)
terms = np.squeeze(np.asarray(df_all))
# print the 'shape' of the matrix - it should indicate the number of unique terms
print(terms.shape)
term_freq_df_all = pd.DataFrame([terms],columns=cvec_all.get_feature_names_out()).transpose() #term_freq_df is with stopwords
term_freq_df_all.columns = ['terms']
# show the first ten words [:10];
# change the values in the brackets to show 30th-40th words [30:40]
# or show the last ten words [:-10]
term_freq_df_all.sort_values(by='terms', ascending=False).iloc[:-10]

(87327,)


,terms
de,108181
van,61552
en,58681
het,55170
in,45629
...,...
ontvangstbevestiging,1
onttrekking,1
boetzelaersborg,1
boetzelaer,1


In [ ]:
# CALCULATE TERM FREQUENCY WITHOUT STOP-WORDS

#cvec_stopped = CountVectorizer(max_df=0.5, token_pattern=r'(?u)\b[A-Za-z]{2,}\b') # max_df could in theory automatically filter stopwords
cvec_stopped = CountVectorizer(stop_words=stopwords_ext, token_pattern=r'(?u)\b[A-Za-z]{2,}\b') # token pattern recognizes only words which are made of letters, and longer than 1 character
cvec_stopped.fit(df0.text) #### CHANGE df0 TO THE DATAFRAME YOU ANALYSE
document_matrix = cvec_stopped.transform(df0.text) #### CHANGE df0 TO THE DATAFRAME YOU ANALYSE
term_batches = np.linspace(0,document_matrix.shape[0],10).astype(int)
i=0
df_stopped = []
while i < len(term_batches)-1:
    batch_result = np.sum(document_matrix[term_batches[i]:term_batches[i+1]].toarray(),axis=0)
    df_stopped.append(batch_result)
    print(term_batches[i+1],"entries' term frequency calculated")
    i += 1

terms_stopped = np.sum(df_stopped,axis=0)
#print(terms_stopped.shape)
term_freq_df_stopped = pd.DataFrame([terms_stopped],columns=cvec_stopped.get_feature_names_out()).transpose()
term_freq_df_stopped.columns = ['terms']
term_freq_df_stopped.sort_values(by='terms', ascending=False).iloc[:10]


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


1216 entries' term frequency calculated
2433 entries' term frequency calculated
3650 entries' term frequency calculated
4867 entries' term frequency calculated
6083 entries' term frequency calculated
7300 entries' term frequency calculated
8517 entries' term frequency calculated
9734 entries' term frequency calculated
10951 entries' term frequency calculated


,terms
kasteel,17702
museum,7290
jaar,4568
onze,4167
uur,3858
landgoed,3806
huis,3506
bezoek,3319
muiderslot,3086
zien,2800


### 1.1 Term frequency for specific terms

In [ ]:
search_word = 'kasteel' # Change this to the word you want to search for

if search_word in term_freq_df_stopped.index: # check if the words exists;
    frequency = term_freq_df_stopped.loc[search_word, 'terms']
    print(f"The word '{search_word}' appears {frequency} times in the current corpus.")
elif search_word in stopwords_ext:
    # If not found in stopped, maybe it was a stop word, so check all terms
    frequency = term_freq_df_all.loc[search_word, 'terms']
    print(f"The word '{search_word}' was filtered out as a stopword. Its total frequency is {frequency} times.")
else:
    print(f"The word '{search_word}' was not found in the corpus.")

### 1.2 TF-IDF vectorization

- What is TF/IDF (term frequency / inverse document frequency)? https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=stopwords_ext, token_pattern=r'(?u)\b[A-Za-z]{2,}\b')
# Fit and transform the text data
tfidf_matrix = vectorizer.fit_transform(df0['text']) #### CHANGE df0 TO THE DATAFRAME YOU ANALYSE
# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
# Add filenames as index
tfidf_df.index = df0['domain'] #### CHANGE df0 TO THE DATAFRAME YOU ANALYSE
# Print the TF-IDF DataFrame
tfidf_df.head()

In [ ]:
# Function to transform the wide TF-IDF DataFrame to a long format
def create_long_tfidf_df_efficiently(tfidf_wide_df):
    data = []
    for domain, row in tfidf_wide_df.iterrows():
        # Get non-zero TF-IDF scores and their corresponding terms to reduce data processing
        active_terms = row[row > 0]
        for term, tfidf_score in active_terms.items():
            data.append({'document': domain, 'term': term, 'tfidf': tfidf_score})
    return pd.DataFrame(data)

# Reorganize the DataFrame from wide to long format using the efficient function
tfidf_df = create_long_tfidf_df_efficiently(tfidf_df)
tfidf_df.head()

In [ ]:
import altair as alt

# Terms in this list will get a red dot in the visualization
term_list = ['kasteel', 'huis', 'children'] # write key terms here

In [ ]:
import altair as alt

# Calculate top 10 TF-IDF terms per domain
top_tfidf_plus = tfidf_df.groupby('document').apply(lambda x: x.nlargest(10, 'tfidf')).reset_index(drop=True)

# Add 'rank' based on tfidf score within each document
top_tfidf_plus['rank'] = top_tfidf_plus.groupby('document')['tfidf'].rank(method='first', ascending=False).astype(int)

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf_plus.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf_plus.shape[0])*0.0001

# Define the base Altair chart
base = alt.Chart(top_tfidf_plusRand).encode(
    x=alt.X('rank:O', axis=alt.Axis(title='Rank (Top 10 Terms)')),
    y=alt.Y('document:N', axis=alt.Axis(title='Domain'))
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tfidf", order="descending")],
    groupby=["document"]
)

# Create the heatmap layer
heatmap = base.mark_rect().encode(
    color=alt.Color('tfidf:Q', scale=alt.Scale(scheme='yellowgreenblue'), title='TF-IDF Score')
)

# Create the text layer with white text
text = base.mark_text(baseline='middle').encode(
    text='term:N',
    color=alt.value('white') # Explicitly set text color to white
)

# Combine the heatmap and text layers and set properties
chart = (heatmap + text).properties(
    title='Top 10 TF-IDF Terms per Domain',
    width=600,
    height=alt.Step(25) # Adjust height based on number of documents
)

chart

In [ ]:
#inspect problems by printing the entire website (document)

document = 'artland.top' # replace with one of the website domains on the left

print("TF-IDF entries for ", document, ": ")
display(tfidf_df[tfidf_df['document'] == document].sort_values(by='tfidf', ascending=False))

print("\nOriginal text entries for", document, ": ")
# Filter the original DataFrame 'df' for the domain 'kasteeltuinen.nl'
domain_text = df0[df0['domain'] == document]['text'].str.cat(sep=' ')
print(domain_text)

### 1.3 Word2Vec model

Vectorizing the corpus with word2vec model
https://en.wikipedia.org/wiki/Word2vec

In [ ]:
!pip install gensim

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
import gensim
from nltk.tokenize import word_tokenize

# X is a list of tokenized texts (i.e. list of lists of tokens)
X = [word_tokenize(item) for item in df0.text.tolist()] # replace df0 with a dataframe you are analysing
#print(X[0:3])
model = gensim.models.Word2Vec(X, min_count=6, vector_size=200) # min_count: how many times a word appears in the corpus; size: number of dimensions

Observe keywords that may be characteristic in the corpus on heritage homes, such as 'castle', 'garden', 'party', 'princess'; try also words related to less obvious themes, like 'servant'

You can ask for 'negative' or 'positive' similarity, and explore how these bring up terms that are opposite to the meaning in a variety of ways.


In [ ]:
model.wv.most_similar(positive=["kasteel"], topn=12)

In [ ]:
model.wv.most_similar(positive=["tuin"], topn=12)

In [ ]:
model.wv.most_similar(negative=["baron"], topn=12)

## 2. Collocations

### 2.1 Analyze specific collocations

In [ ]:
# define vectorization functions

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from itertools import islice

# SCI-KIT method, produces lists of co-occurencies for specific terms
def vectorize_text(df, column):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df[column])
    return X, vectorizer

def find_collocations(text, target_words):
    words = text.split()
    collocations = []
    for i in range(len(words) - 1):
        if words[i] in target_words:
            collocations.append((words[i], words[i + 1]))
        if words[i + 1] in target_words:
            collocations.append((words[i + 1], words[i]))
    return collocations

def get_frequent_collocations(df, column, most_frequent_words):
    collocations = []
    for text_content in df[column]:
        collocations.extend(find_collocations(text_content, most_frequent_words))
    collocation_counts = Counter(collocations)
    frequent_collocations = {}
    for word in most_frequent_words:
        word_collocations = {collocation: count for collocation, count in collocation_counts.items() if word in collocation}
        frequent_collocations[word] = dict(islice(Counter(word_collocations).most_common(20), 20)) # change these two values to get more or less terms
    return frequent_collocations

def analyze_word_collocations(df, column):
    X, vectorizer = vectorize_text(df, column)
    most_frequent_words = search_words
    frequent_collocations = get_frequent_collocations(df, column, most_frequent_words)
    return frequent_collocations

Define the search term here, analyse whether it appears in the corpus and next to which words (excluding stopwords)

In [ ]:
# search for words from this list or use another list
search_words = ['kasteel']

In [ ]:
collocations = analyze_word_collocations(df0, 'text') # CHANGE df0 TO DATAFRAME YOU ARE ANALYSING

In [ ]:
data = []
for word, colloc_dict in collocations.items():
   for collocation, count in colloc_dict.items():
       #collocation_str = ' '.join(collocation)  # Join collocation words into a single string
       data.append([word, collocation[1], count])
collocations_df = pd.DataFrame(data, columns=['Word', 'Collocation', 'Count'])
print(collocations_df.to_markdown(index=True))

#### 2.1.1. Analyze term frequency and collocation in page titles

In [ ]:
# add a column 'page_title' to th dataframe (df0 or df1-3) extracting the value of 'title' key in metadata dictionary in each entry (lambda function)

df0['page_title'] = df0['metadata'].apply(lambda x: x.get('title'))
df0['page_title'].head()

In [ ]:
search_term = 'baron'
total_occurrences = 0

print("Searching for", search_term, ":")
for index, title in df0['page_title'].items():
    if isinstance(title, str):
        # Convert to lowercase for case-insensitive search
        title_lower = title.lower()
        # Count occurrences in the current title
        occurrences_in_title = title_lower.count(search_term)
        if occurrences_in_title > 0:
            print(title)
            total_occurrences += occurrences_in_title

print("\nTotal occurrences of", search_term, "across all titles: ", total_occurrences)

In [ ]:
# search for words from this list or use another list
search_words = ['baron', 'kasteel']

In [ ]:
# compute the collocations in titles
collocations = analyze_word_collocations(df0, 'page_title') # CHANGE df0 TO DATAFRAME YOU ARE ANALYSING

In [ ]:
data = []
for word, colloc_dict in collocations.items():
   for collocation, count in colloc_dict.items():
       #collocation_str = ' '.join(collocation)  # Join collocation words into a single string
       data.append([word, collocation[1], count])
collocations_df = pd.DataFrame(data, columns=['Word', 'Collocation', 'Count'])
print(collocations_df.to_markdown(index=True))

### 2.2 Analyse collocations in sentences

In [ ]:
#function to remove non-ascii characters
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

In [ ]:
# import the advanced Natural Language Processing (NLP) library
# which we will use to analze the grammatical structure
import spacy

In [ ]:
# download the suitable language pipeline
# Dutch: nl_core_news_sm
# French: fr_core_news_sm
# German: de_core_news_sm
# English: en_core_web_sm (available by default)
!python -m spacy download nl_core_news_sm

In [ ]:
nlp = spacy.load('nl_core_news_sm') # change to FR/DE/EN code module, see names above

Lemmatize the text in 'text' column of the dataframe
https://en.wikipedia.org/wiki/Lemmatization

In [ ]:
import string

#function to clean and lemmatize text
def clean_documents(text):
    # Ensure text is a string to prevent errors with non-string inputs
    text = str(text)

    # Remove all non-alphabetic characters (incl. punctuation, numbers, etc.)
    cleaned_text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # Replace multiple spaces with a single space and strip leading/trailing spaces.
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    # If the text becomes empty after cleaning (e.g., if it was only punctuation), return an empty list
    if not cleaned_text:
        return []

    # lemmatize the text with spacy
    doc = nlp(cleaned_text, disable=['parser','ner'])
    # Get lemmas, convert to lowercase, and filter out any empty strings that might arise
    # token.lemma_.strip() ensures no whitespace-only strings are included
    lemma = [token.lemma_.lower() for token in doc if token.lemma_.strip()]
    return lemma

In [ ]:
# Lemmatize the text in .text column
lemmatized = df0.text.map(clean_documents) # CHANGE df0 TO THE DATAFRAME YOU ARE ANALYSING
lemmatized.head()

In [ ]:
# Flatten the list of lists into one list + remove stopwords
unlist_documents = [word for sublist in lemmatized for word in sublist if word not in stopwords_ext]

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
# initiate bigrams and trigrams
bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

In [ ]:
# identify all collocations in the flat list of words from all documents
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_documents)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_documents)

Calculate basic frequency

In [ ]:
bigram_freq = bigramFinder.ngram_fd.items()

In [ ]:
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)

In [ ]:
bigramFreqTable.head().reset_index(drop=True)

In [ ]:
# compute basic trigrams frequency
trigram_freq = trigramFinder.ngram_fd.items()
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)
trigramFreqTable[:10]

Search for a specific term in bigram & trigram frequency table:

In [ ]:
search_term = 'kasteel'

In [ ]:
filtered_bigrams_for_term = bigramFreqTable[bigramFreqTable['bigram'].apply(lambda x: search_term in x)]

display(filtered_bigrams_for_term.head(10))

In [ ]:
filtered_trigrams_for_term = trigramFreqTable[trigramFreqTable['trigram'].apply(lambda x: search_term in x)]

display(filtered_trigrams_for_term.head(10))

#### 2.2.1 Meaningful bi- and tri-grams

Identify birgrams and trigramms by filtering adjectives and nouns based on an nltk functionality

In [ ]:
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    for word in ngram:
        if word in stopwords_ext:
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [ ]:
#filter bigrams

### WARNING: THIS MAY TAKE A LONGER TIME TO COMPLETE (~3min)
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]

In [ ]:
filtered_bi[:10]

In [ ]:
#filter trigrams

### WARNING: THIS MAY TAKE A LONGER TIME TO COMPLETE (~3min)
filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda x: rightTypes(x))]

In [ ]:
filtered_tri[:10]

Search for a specific term in FILTERED bigram & trigram frequency tables:


In [ ]:
search_term = 'kasteel'

In [ ]:
filtered_bigrams_for_term = filtered_bi[filtered_bi['bigram'].apply(lambda x: search_term in x)]

display(filtered_bigrams_for_term.head(10))

In [ ]:
filtered_trigrams_for_term = filtered_tri[filtered_tri['trigram'].apply(lambda x: search_term in x)]

display(filtered_trigrams_for_term.head(10))

Use advanced statistical methods like the Chi-Square to identify meaninful collocations
https://en.wikipedia.org/wiki/Chi-squared_test

In [ ]:
# filter bigrams using chi-square
bigramChiTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','chi-sq']).sort_values(by='chi-sq', ascending=False)
bigramChiTable.head()

In [ ]:
# find meaningful trigrams by filtering basic frequency table
# function to filter trigrams
def rightTypesTri(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or '  ' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in stopwords_ext:
            return False
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in first_type and tags[2][1] in third_type:
        return True
    else:
        return False

In [ ]:
# Chi-sqare frequency calculation for trigrams
trigramChiTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.chi_sq)), columns=['trigram','chi-sq']).sort_values(by='chi-sq', ascending=False)
trigramChiTable.head(10)

## 3. Topic Modelling

In [ ]:
import sklearn.feature_extraction.text as text

# min_df: ignore words occurring in fewer than `n` documents
# stop_words: ignore very common words ("the", "and", "or", "to", ...)
vec = text.CountVectorizer(lowercase=True, min_df=100, stop_words=stopwords_ext)
dtm = vec.fit_transform(df0['text']) # CHANGE df0 TO THE DATAFRAME YOU ARE ANALYSING

In [ ]:
print(f'Shape of document-term matrix: {dtm.shape}. '
      f'Number of tokens {dtm.sum()}')

Shape of document-term matrix: (10951, 1449). Number of tokens 667527


In [ ]:
import sklearn.decomposition as decomposition
NUM_TOPICS = 10
lda_model = decomposition.LatentDirichletAllocation(
    n_components=NUM_TOPICS, learning_method='online', random_state=1)
lda_Z = lda_model.fit_transform(dtm)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)

document_topic_distributions = lda_model.fit_transform(dtm)

top_n=10
for idx, topic in enumerate(lda_model.components_):
  print("Topic %d:" % (idx))
  print([(vec.get_feature_names_out()[i], topic[i])
  for i in topic.argsort()[:-top_n - 1:-1]])

(10951, 10)
Topic 0:
[('kasteel', np.float64(4101.640288488984)), ('arcen', np.float64(2672.781493046379)), ('kasteeltuinen', np.float64(1694.7489201496946)), ('park', np.float64(1397.0720569350326)), ('slot', np.float64(1035.0133196429865)), ('zuylen', np.float64(736.943361875806)), ('loevestein', np.float64(729.5426883199992)), ('schlossgärten', np.float64(700.957166941864)), ('heeswijk', np.float64(674.5268436303486)), ('tuinen', np.float64(600.2164955599784))]
Topic 1:
[('kasteel', np.float64(5343.903246167998)), ('huis', np.float64(2646.805228065219)), ('gevonden', np.float64(1162.9475621763222)), ('museum', np.float64(1021.3593938902801)), ('bergh', np.float64(904.3967183847965)), ('nienoord', np.float64(899.6456678324004)), ('doorn', np.float64(767.9606358397521)), ('amerongen', np.float64(717.9832861250422)), ('dekema', np.float64(691.3891992164619)), ('online', np.float64(511.50544201251256))]
Topic 2:
[('castle', np.float64(2040.8779141280015)), ('cookies', np.float64(1337.83

In [ ]:
no_top_words = 12
no_top_documents = 5
lda_H = lda_model.components_
tf_feature_names = vec.get_feature_names_out()

def display_topics(H, Z, feature_names, docs, no_top_words, no_top_documents):
    for idx, topic in enumerate(H):
        print("Topic %d:" % (idx))
        print("KEYWORDS", " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( Z[:,
                                        idx] )[::-1][0:no_top_documents]
        # good for checking which documents are the most characteristic for certain topics
        for doc_index in top_doc_indices:
            print("TOP DOCS", docs[doc_index])

display_topics(lda_H, lda_Z, tf_feature_names, df0['url'].tolist(), no_top_words, no_top_documents)

Topic 0:
KEYWORDS kasteel arcen kasteeltuinen park slot zuylen loevestein schlossgärten heeswijk tuinen jullie gardens
TOP DOCS http://slotzuylen.nl/locatie/slot-zuylen/
TOP DOCS https://www.slotloevestein.nl/en/contact/
TOP DOCS https://kasteelteylingen.nl/bestuur/
TOP DOCS https://www.kasteeltuinen.nl/de/events/geschenkidee-fuer-ihre-mitarbeiter/
TOP DOCS https://www.kasteeltuinen.nl/de/die-schoenste-sehenswuerdigkeit-in-arcen/
Topic 1:
KEYWORDS kasteel huis gevonden museum bergh nienoord doorn amerongen dekema online bezoek doorwerth
TOP DOCS https://www.huisdoorn.nl/ontdek-het-museum/huis-en-park/
TOP DOCS https://www.huisdoorn.nl/educatie/
TOP DOCS https://www.huisdoorn.nl/en/discover-the-museum/house-and-park/emperor-wilhelm-ii/princess-hermine/
TOP DOCS https://www.kasteelamerongen.nl/over-het-kasteel/geschiedenis/
TOP DOCS https://www.huisdoorn.nl/de/ueber-uns/unterstuetzen-sie-uns/
Topic 2:
KEYWORDS castle cookies museum wij contact stichting vragen privacy informatie gegevens